Best For :

TYPE 1 DATASET

1. Logistic regression
**( C=0.1 , l1_ratio = 0.4 , penalty='elasticnet' , solver='saga' , tol=0.12 ,max_iter=1000, class_weight='balanced')**

2. SVM
**( C=1.1,coef0=0,degree=8,gamma='scale', kernel='rbf',tol=0.47,class_weight='balanced' )**

3. XGBoost
**( 'booster': 'gbtree', 'colsample_bylevel': 0.5, 'colsample_bynode': 0.5, 'colsample_bytree': 0.8, 'gamma': 0.2, 'grow_policy': 'depthwise',
  'learning_rate': 0.6, 'max_delta_step': 0.41, 'max_depth': 5, 'min_child_weight': 0.42, 'num_parallel_tree': 1, 'reg_alpha': 0.1, 'reg_lambda': 0.45,
  'subsample': 0.8, 'tree_method': 'exact')**

Best For :

TYPE 2 DATASET

1. Logistic regression
**( C=0.45  , penalty='l2' , solver='saga' , tol=0.125 , max_iter=1000, class_weight='balanced')**

2. SVM
**( C=2.4,coef0=0.06,degree=7,gamma='scale', kernel='rbf',tol=0.38,class_weight='balanced' )**

3. XGBoost
**( 'booster': 'gbtree', 'colsample_bylevel': 0.5, 'colsample_bynode': 0.5, 'colsample_bytree': 0.8, 'gamma': 0.2, 'grow_policy': 'depthwise',
  'learning_rate': 0.7, 'max_delta_step': 0.41, 'max_depth': 8, 'min_child_weight': 0.42, 'num_parallel_tree': 1, 'reg_alpha': 2.3, 'reg_lambda': 2.3,
  'subsample': 0.8, 'tree_method': 'exact')**



In [1]:
# warning disable
import warnings
warnings.filterwarnings("ignore")

def pped(prob_matrx, threshold):
    return (prob_matrx[:, 1] >= threshold).astype(int)

In [8]:
client.close()

In [2]:
import logging

from dask.distributed import Client
from dask import delayed, compute

client = Client(threads_per_worker=2, n_workers=4,silence_logs=logging.ERROR)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 11.57 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:45455,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 11.57 GiB
Comm: tcp://127.0.0.1:38719,Total threads: 2
Dashboard: http://127.0.0.1:33721/status,Memory: 2.89 GiB
Nanny: tcp://127.0.0.1:34161,


### We will be experimenting with the following:

Logistic Regression (JP)
Naive Bayes (SP)
Ensemble Methods (Tree methods) (SK)
XGBoost (SA)
SVM (SA)
Multilayer Perceptron *


In [3]:
import numpy as np
import pandas as pd
from helpers import get_train_test, RANDOM_STATE
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.metrics import (
    roc_auc_score,
    f1_score,
    accuracy_score,
    precision_score,
    recall_score,
    confusion_matrix
)

In [4]:
df_1 = pd.read_csv('../dataset/processed_dataset/final_data_S1.csv')
#
df_2 = pd.read_csv('../dataset/processed_dataset/final_data_S2.csv')

## Working Type 1 Dataprocessed Dataset

In [6]:

X_train_1, X_test_1, y_train_1, y_test_1 = get_train_test(df_1)


In [7]:
print("== Training Values ==")
print(y_train_1.value_counts(normalize=True))
print('\n')
print("== Testing Values ==")
print(y_test_1.value_counts(normalize=True))

== Training Values ==
1    0.75
0    0.25
Name: class, dtype: float64


== Testing Values ==
1    0.75
0    0.25
Name: class, dtype: float64


In [11]:
params = {
    'threshold': [x/100 for x in range(30, 72, 1)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(len(list(param_dict)))

42


In [15]:

@delayed
def run_model(param, X_train, y_train, X_test, y_test):
    model_obj = LogisticRegression(C=0.1 , l1_ratio = 0.4 , penalty='elasticnet' , solver='saga' , tol=0.12 ,max_iter=1000, class_weight='balanced', random_state=RANDOM_STATE)
    model_obj.fit(X_train, y_train)
    predictions = pped(model_obj.predict_proba(X_test), param['threshold'])
    param.update({
        "accuracy_score":accuracy_score(y_test, predictions),
        "f1_score":f1_score(y_test, predictions),
        "precision_score":precision_score(y_test, predictions),
        "recall_score":recall_score(y_test, predictions),
        "roc_auc_score":roc_auc_score(y_test, predictions),
        "confusion_matrix":str(confusion_matrix(y_test, predictions))
    })
    return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)

In [16]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
# #
# # not_used = ['random_state','class_weight']
# # final.drop(not_used,axis=1,inplace=
final

,threshold,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,confusion_matrix
0,0.30,0.757353,0.860759,0.755556,1.000000,0.514706,[[ 1 33]\n [ 0 102]]
1,0.31,0.757353,0.860759,0.755556,1.000000,0.514706,[[ 1 33]\n [ 0 102]]
2,0.32,0.764706,0.864407,0.761194,1.000000,0.529412,[[ 2 32]\n [ 0 102]]
3,0.33,0.764706,0.863248,0.765152,0.990196,0.539216,[[ 3 31]\n [ 1 101]]
4,0.34,0.764706,0.863248,0.765152,0.990196,0.539216,[[ 3 31]\n [ 1 101]]
5,0.35,0.772059,0.866953,0.770992,0.990196,0.553922,[[ 4 30]\n [ 1 101]]
6,0.36,0.779412,0.870690,0.776923,0.990196,0.568627,[[ 5 29]\n [ 1 101]]
7,0.37,0.772059,0.865801,0.775194,0.980392,0.563725,[[ 5 29]\n [ 2 100]]
8,0.38,0.779412,0.869565,0.781250,0.980392,0.578431,[[ 6 28]\n [ 2 100]]
9,0.39,0.779412,0.869565,0.781250,0.980392,0.578431,[[ 6 28]\n [ 2 100]]


In [ ]:
final.to_csv('param/final_data_S1_logistic_regression-test2.csv',index=False)

In [ ]:

params = {
    'C':[x/10 for x in range(1,30,2)],
    'kernel':['rbf'],
    'degree':[x for x in range(5,9,1)],
    'gamma':['scale'],
    'coef0':[x/100 for x in range(0,26,2)],
    'tol':[x/100 for x in range(25,50,2)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(len(list(param_dict)))

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = SVC(**param, class_weight='balanced',
                    random_state=RANDOM_STATE)
    model_obj.fit(X_train, y_train)
    param.update({
        "accuracy_score":accuracy_score(y_test, predictions),
        "f1_score":f1_score(y_test, predictions),
        "precision_score":precision_score(y_test, predictions),
        "recall_score":recall_score(y_test, predictions),
        "roc_auc_score":roc_auc_score(y_test, predictions),
        "confusion_matrix":str(confusion_matrix(y_test, predictions))
    })
    return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)


In [ ]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

In [ ]:
final.to_csv('param/final_data_S1_svm-test4.csv',index=False)

In [16]:
from helpers import XGBoostModel

# model_obj = XGBoostModel(
#     **list(param_dict)[100]
# )
# model_obj.fit(X_train_1, y_train_1, X_test_1, y_test_1, verbose=True)

In [17]:
[x/100 for x in range(10,100,25)]

[0.1, 0.35, 0.6, 0.85]

In [34]:
from helpers import XGBoostModel

# n_estimators = 500
# importance_type = 'gain'
# eval_metric = ['auc','logloss']
# early_stopping_rounds = 20
# scale_pos_weight = pos_ratio(y_train_1)
# max_leaves = 0
# objective = "binary:logistic"

# tree_method = 'gpu_hist' #gpu
colsample_bytree = 0.5
colsample_bylevel = 0.5
colsample_bynode = 0.5
# max_depth = 4
# learning_rate = 0.1
# subsample = 0.5
# max_delta_step = 0.5
# num_parallel_tree = 1

params = {
    'max_depth':[5],
    'grow_policy':['depthwise'],
    'learning_rate':[0.6],
    'booster':['gbtree'],
    'gamma':[0.2], #min_split_loss
    'min_child_weight':[0.42],
    'max_delta_step':[0.41],
    'reg_alpha':[0.1],
    'reg_lambda':[0.45],
    'num_parallel_tree':[1],
    'tree_method':['exact'],
    'subsample':[0.8],
    'colsample_bytree':[0.8],
    'colsample_bylevel':[0.5],
    'colsample_bynode':[0.5]
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(param_dict.__len__())

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = XGBoostModel(
        **param,
    )
    model_obj.fit(X_train, y_train, X_test, y_test, verbose=False)
    param.update({
        "accuracy_score":accuracy_score(y_test, predictions),
        "f1_score":f1_score(y_test, predictions),
        "precision_score":precision_score(y_test, predictions),
        "recall_score":recall_score(y_test, predictions),
        "roc_auc_score":roc_auc_score(y_test, predictions),
        "confusion_matrix":str(confusion_matrix(y_test, predictions)),
        "best_logloss":model_obj.score(),
        "best_iteration":model_obj.model.best_iteration
    })
    return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_1, y_train_1, X_test_1, y_test_1) for each in param_dict)


1


In [36]:
list(param_dict)

[{'booster': 'gbtree',
  'colsample_bylevel': 0.5,
  'colsample_bynode': 0.5,
  'colsample_bytree': 0.8,
  'gamma': 0.2,
  'grow_policy': 'depthwise',
  'learning_rate': 0.6,
  'max_delta_step': 0.41,
  'max_depth': 5,
  'min_child_weight': 0.42,
  'num_parallel_tree': 1,
  'reg_alpha': 0.1,
  'reg_lambda': 0.45,
  'subsample': 0.8,
  'tree_method': 'exact'}]

In [35]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

,booster,colsample_bylevel,colsample_bynode,colsample_bytree,gamma,grow_policy,learning_rate,max_delta_step,max_depth,min_child_weight,...,subsample,tree_method,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,confusion_matrix,best_logloss,best_iteration
0,gbtree,0.5,0.5,0.8,0.2,depthwise,0.6,0.41,5,0.42,...,0.8,exact,0.911765,0.942857,0.916667,0.970588,0.852941,[[25 9]\n [ 3 99]],0.3140002019512501,30


In [39]:
client.close()

Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGB

In [23]:
final.to_csv('param/final_data_S1_xgboost-test5.csv',index=False)

## Working Type 2 Dataprocessed Dataset

In [10]:
y_2 = df_2['class']
X_2 = df_2.drop(['class'], axis=1)
X_train_2, X_test_2, y_train_2, y_test_2 = get_train_test(X_2, y_2)


In [11]:
print("== Training Values ==")
print(y_train_2.value_counts(normalize=True))
print('\n')
print("== Testing Values ==")
print(y_test_2.value_counts(normalize=True))

== Training Values ==
1    0.75
0    0.25
Name: class, dtype: float64


== Testing Values ==
1    0.75
0    0.25
Name: class, dtype: float64


In [20]:
params = {
    'penalty':['l2','elasticnet'],
    'tol':[x/1000 for x in range(100, 250, 10)],
    'C':[x/100 for x in range(10, 50, 5)],
    'solver':['saga'],
    'l1_ratio':[x/10 for x in range(2, 5, 1)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(len(list(param_dict)))

@delayed
def run_model(param, X_train, y_train, X_test, y_test):
    if check_param(param):
        model_obj = LogisticRegression(**param, max_iter=1000,
                                       class_weight='balanced', random_state=RANDOM_STATE)
        model_obj.fit(X_train, y_train)
        param.update({
            "accuracy_score":accuracy_score(y_test, predictions),
            "f1_score":f1_score(y_test, predictions),
            "precision_score":precision_score(y_test, predictions),
            "recall_score":recall_score(y_test, predictions),
            "roc_auc_score":roc_auc_score(y_test, predictions),
            "confusion_matrix":str(confusion_matrix(y_test, predictions))
        })
        return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_2, y_train_2, X_test_2, y_test_2) for each in param_dict)

720


In [ ]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

In [22]:
final.to_csv('param/final_data_S2_logistic_regression-test2.csv',index=False)

In [44]:
params = {
    'C':[x/10 for x in range(10,30,2)],
    'kernel':['rbf'],
    'degree':[1,3,5,7,9],
    'gamma':['scale'],
    'coef0':[x/100 for x in range(0,26,2)],
    'tol':[x/100 for x in range(20,40,2)],
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(param_dict.__len__())

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = SVC(**param, class_weight='balanced',
                    random_state=RANDOM_STATE)
    model_obj.fit(X_train, y_train)
    param.update({
        "accuracy_score":accuracy_score(y_test, predictions),
        "f1_score":f1_score(y_test, predictions),
        "precision_score":precision_score(y_test, predictions),
        "recall_score":recall_score(y_test, predictions),
        "roc_auc_score":roc_auc_score(y_test, predictions),
        "confusion_matrix":str(confusion_matrix(y_test, predictions))
    })
    return pd.DataFrame(param,index=[0])

dfs = (run_model(each , X_train_2, y_train_2, X_test_2, y_test_2) for each in param_dict)


6500


In [45]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

,C,coef0,degree,gamma,kernel,tol,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,confusion_matrix
0,1.0,0.00,1,scale,rbf,0.20,0.823529,0.880000,0.897959,0.862745,0.784314,[[24 10]\n [14 88]]
1,1.0,0.00,1,scale,rbf,0.22,0.823529,0.880000,0.897959,0.862745,0.784314,[[24 10]\n [14 88]]
2,1.0,0.00,1,scale,rbf,0.24,0.830882,0.884422,0.907216,0.862745,0.799020,[[25 9]\n [14 88]]
3,1.0,0.00,1,scale,rbf,0.26,0.830882,0.884422,0.907216,0.862745,0.799020,[[25 9]\n [14 88]]
4,1.0,0.00,1,scale,rbf,0.28,0.816176,0.874372,0.896907,0.852941,0.779412,[[24 10]\n [15 87]]
...,...,...,...,...,...,...,...,...,...,...,...,...
6495,2.8,0.24,9,scale,rbf,0.30,0.882353,0.922330,0.913462,0.931373,0.833333,[[25 9]\n [ 7 95]]
6496,2.8,0.24,9,scale,rbf,0.32,0.889706,0.927536,0.914286,0.941176,0.838235,[[25 9]\n [ 6 96]]
6497,2.8,0.24,9,scale,rbf,0.34,0.897059,0.932692,0.915094,0.950980,0.843137,[[25 9]\n [ 5 97]]
6498,2.8,0.24,9,scale,rbf,0.36,0.875000,0.917073,0.912621,0.921569,0.828431,[[25 9]\n [ 8 94]]


In [46]:
final.to_csv('param/final_data_S2_svm-test2.csv',index=False)

In [ ]:
[x/100 for x in range(10,50,4)]

In [ ]:
from collections import Counter
from xgboost import XGBClassifier
def pos_ratio(Y) -> float:
    """
    Compute the ratio of positive class in the dataset.
    """
    counter = Counter(Y)
    sum_wpos = counter[1]
    sum_wneg = counter[0]

    return sum_wneg / sum_wpos

In [ ]:
model_obj = XGBClassifier(
    **list(param_dict)[100],use_label_encoder=False, random_state=RANDOM_STATE, objective = "binary:logistic",
    n_estimators = 500, max_leaves = 0,  tree_method = 'hist',
    colsample_bytree = 0.5,colsample_bylevel = 0.5,colsample_bynode = 0.5,scale_pos_weight = pos_ratio(y_train_1),
    importance_type = 'gain',max_depth = 4,
    learning_rate = 0.1 ,subsample = 0.5,max_delta_step = 0.5 ,num_parallel_tree = 1
)
model_obj.fit(X_train_1, y_train_1, eval_set=[(X_test_1, y_test_1)], eval_metric=eval_metric, early_stopping_rounds=early_stopping_rounds)

In [1]:
[x/10 for x in range(0,50,5)]

[0.0, 0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.0, 4.5]

In [72]:
from helpers import XGBoostModel

# n_estimators = 500
# importance_type = 'gain'
# eval_metric = ['auc','logloss']
# early_stopping_rounds = 20
# scale_pos_weight = pos_ratio(y_train_1)
# max_leaves = 0
# objective = "binary:logistic"

# tree_method = 'gpu_hist' #gpu
# colsample_bytree = 0.5
# colsample_bylevel = 0.5
# colsample_bynode = 0.5
# max_depth = 4
# learning_rate = 0.1
# subsample = 0.5
# max_delta_step = 0.5
# num_parallel_tree = 1

params = {
    'max_depth':[8],
    'grow_policy':['depthwise'],
    'learning_rate':[0.7],
    'booster':['gbtree'],
    'gamma':[0.2], #min_split_loss
    'min_child_weight':[0.42],
    'max_delta_step':[0.41], #max_delta_step
    'reg_alpha':[2.3],
    'reg_lambda':[2.3],
    'num_parallel_tree':[1],
    'subsample':[0.8],
    'tree_method':['exact']
}

from sklearn.model_selection import ParameterGrid
param_dict = ParameterGrid(params)
print(param_dict.__len__())

@delayed
def run_model(param, X_train, y_train, X_test, y_test):

    model_obj = XGBoostModel(
        **param,colsample_bytree=0.8,colsample_bylevel=0.5,colsample_bynode=0.5,
    )
    model_obj.fit(X_train, y_train, X_test, y_test, verbose=False)
    param.update({
        "accuracy_score":accuracy_score(y_test, predictions),
        "f1_score":f1_score(y_test, predictions),
        "precision_score":precision_score(y_test, predictions),
        "recall_score":recall_score(y_test, predictions),
        "roc_auc_score":roc_auc_score(y_test, predictions),
        "confusion_matrix":str(confusion_matrix(y_test, predictions)),
        "best_logloss":model_obj.score(),
        "best_iteration":model_obj.model.best_iteration
    })
    return pd.DataFrame(param,index=[0])

# create dask parallel function to run xgb
dfs = (run_model(each, X_train_2, y_train_2, X_test_2, y_test_2) for each in param_dict)


1


In [73]:
#
value = compute(dfs,num_workers=4)
#
final = pd.concat(value[0])
value = []
final.reset_index(drop=True,inplace=True)
#
# not_used = ['random_state','class_weight']
# final.drop(not_used,axis=1,inplace=
final

,booster,gamma,grow_policy,learning_rate,max_delta_step,max_depth,min_child_weight,num_parallel_tree,reg_alpha,reg_lambda,subsample,tree_method,accuracy_score,f1_score,precision_score,recall_score,roc_auc_score,confusion_matrix,best_logloss,best_iteration
0,gbtree,0.2,depthwise,0.7,0.41,8,0.42,1,2.3,2.3,0.8,exact,0.882353,0.923077,0.90566,0.941176,0.823529,[[24 10]\n [ 6 96]],0.3245291632072836,49


In [33]:
final.to_csv('param/final_data_S2_xgboost-test4.csv',index=False)

In [34]:
client.close()

Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGBoost model...
Training XGB